In [7]:
# !pip3 install --force-reinstall git+https://github.com/fra31/auto-attack

In [1]:
cd /scratch/snx3000/pchaudha/Shravan/FML/SelfSupDefense/ 

/scratch/snx3000/pchaudha/Shravan/FML/SelfSupDefense


In [2]:
import argparse
import logging

''''If you see bad robustness, it is due to use the wrong normalization in L26-32'''
import sys
import math, time
import random

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import os

from learning.wideresnet import WideResNet, WideResNetBD, WideResNetMed_SSL, WRN34_out_branch
from learning.preactresnet import PreActResNet18Mhead, Res18_out3_model,Res18_out4_model,Res18_out5_model,Res18_out6_model
from data.gaussian_blur import GaussianLayer

from utils import *
from cifar10_defense_rebAA import *
from Qmodels.wide_resnet import *
from Qmodels.utils import *

head num 8


In [3]:
mu = torch.tensor(cifar10_mean).view(3,1,1).cuda()
std = torch.tensor(cifar10_std).view(3,1,1).cuda()

In [4]:
def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--model', default='WideResNet')  #WideResNet
    parser.add_argument('--l2', default=0, type=float)
    parser.add_argument('--l1', default=0, type=float)
    parser.add_argument('--batch-size', default=1024, type=int)
    parser.add_argument('--contrastive_bs', default=512, type=int)
    parser.add_argument('--data-dir', default='../cifar-data', type=str)
    parser.add_argument('--epochs', default=200, type=int)
    parser.add_argument('--lr-schedule', default='piecewise', choices=['superconverge', 'piecewise', 'linear', 'piecewisesmoothed', 'piecewisezoom', 'onedrop', 'multipledecay', 'cosine'])
    parser.add_argument('--lr-max', default=0.1, type=float)
    parser.add_argument('--lr-one-drop', default=0.01, type=float)
    parser.add_argument('--lam_res', default=1, type=float)
    parser.add_argument('--adda_times', default=1, type=float)
    parser.add_argument('--lr-drop-epoch', default=100, type=int)
    parser.add_argument('--attack', default='pgd', type=str, choices=['pgd', 'fgsm', 'free', 'none'])
    parser.add_argument('--epsilon', default=8, type=int)
    parser.add_argument('--attack-iters', default=10, type=int)
    parser.add_argument('--bd_attack_iters', default=4, type=int)
    parser.add_argument('--restarts', default=1, type=int)
    parser.add_argument('--neg_size', default=10, type=int)
    parser.add_argument('--pgd-alpha', default=2, type=float)
    parser.add_argument('--fgsm-alpha', default=1.25, type=float)
    parser.add_argument('--norm', default='l_inf', type=str, choices=['l_inf', 'l_2', 'l_1'])
    parser.add_argument('--fgsm-init', default='random', choices=['zero', 'random', 'previous'])
    parser.add_argument('--fname', default='train_ssl', type=str)
    import socket
    # if 'cv10' in socket.gethostname():
    #     parser.add_argument('--save_root_path', default='/local/vondrick/chengzhi/SSRobust', type=str)
    if 'cv' in socket.gethostname():
        parser.add_argument('--save_root_path', default='/scratch/snx3000/pchaudha/Shravan/FML/SelfSupDefense/', type=str)
    else:
        parser.add_argument('--save_root_path', default='/scratch/snx3000/pchaudha/Shravan/FML/SelfSupDefense/', type=str)

    parser.add_argument('--ssl_model_path', default='', type=str)
    parser.add_argument('--attack_type', default='', type=str)
    parser.add_argument('--seed', default=0, type=int)
    parser.add_argument('--half', action='store_true')
    parser.add_argument('--width-factor', default=10, type=int)
    parser.add_argument('--constrastive_head', default=16, type=int)
    parser.add_argument('--md_path', default='', type=str)
    parser.add_argument('--cutout', action='store_true')
    parser.add_argument('--cutout-len', type=int)
    parser.add_argument('--mixup', action='store_true')
    parser.add_argument('--rand', action='store_true')

    parser.add_argument('--debug', action='store_true')
    parser.add_argument('--MCtimes', default=1, type=int)
    parser.add_argument('--n_views', default=4, type=int)
    parser.add_argument('--eval_freq', default=10, type=int)
    parser.add_argument('--mixup-alpha', type=float)
    parser.add_argument('--eval', action='store_true')
    parser.add_argument('--val', action='store_true')
    parser.add_argument('--carmon', action='store_true')
    parser.add_argument('--TRADES', action='store_true')
    parser.add_argument('--Bag', action='store_true')
    parser.add_argument('--res18', action='store_true')
    parser.add_argument('--new', action='store_true')
    parser.add_argument('--eval_only', action='store_true')
    parser.add_argument('--random_noise', action='store_true')
    parser.add_argument('--foolbox', action='store_true')
    return parser.parse_known_args()

In [5]:
print("eval only")
adda_times=1

args, _ = get_args()
import uuid
import datetime
unique_str = str(uuid.uuid4())[:8]
timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d_%H:%M:%S')

args.fname = os.path.join(args.save_root_path, args.fname, timestamp + unique_str)

args.fname = 'test'
args.md_path = '/scratch/snx3000/pchaudha/Shravan/FML/SelfSupDefense/Qmodels/model_wide_cifar10.pth'
args.carmon = True
args.eval_only = True
args.ssl_model_path = '/scratch/snx3000/pchaudha/Shravan/FML/SelfSupDefense/checkpoints/ssl_model_130.pth'
args.contrastive_bs = 128
args.batch_size = 128


if not os.path.exists(args.fname):
    os.makedirs(args.fname)

logger = logging.getLogger(__name__)
logging.basicConfig(
    format='[%(asctime)s] - %(message)s',
    datefmt='%Y/%m/%d %H:%M:%S',
    level=logging.DEBUG,
    handlers=[
        logging.FileHandler(os.path.join(args.fname, 'eval.log' if args.eval else 'output.log')),
        logging.StreamHandler()
    ])

logger.info(args)

[2022/05/04 01:41:52] - Namespace(model='WideResNet', l2=0, l1=0, batch_size=128, contrastive_bs=128, data_dir='../cifar-data', epochs=200, lr_schedule='piecewise', lr_max=0.1, lr_one_drop=0.01, lam_res=1, adda_times=1, lr_drop_epoch=100, attack='pgd', epsilon=8, attack_iters=10, bd_attack_iters=4, restarts=1, neg_size=10, pgd_alpha=2, fgsm_alpha=1.25, norm='l_inf', fgsm_init='random', fname='test', save_root_path='/scratch/snx3000/pchaudha/Shravan/FML/SelfSupDefense/', ssl_model_path='/scratch/snx3000/pchaudha/Shravan/FML/SelfSupDefense/checkpoints/ssl_model_130.pth', attack_type='', seed=0, half=False, width_factor=10, constrastive_head=16, md_path='/scratch/snx3000/pchaudha/Shravan/FML/SelfSupDefense/Qmodels/model_wide_cifar10.pth', cutout=False, cutout_len=None, mixup=False, rand=False, debug=False, MCtimes=1, n_views=4, eval_freq=10, mixup_alpha=None, eval=False, val=False, carmon=True, TRADES=False, Bag=False, res18=False, new=False, eval_only=True, random_noise=False, foolbox=Fa

eval only


In [6]:
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)

transforms = [Crop(32, 32), FlipLR()]
dataset = cifar10(args.data_dir)

train_set = list(zip(transpose(pad(dataset['train']['data'], 4) / 255.),
                     dataset['train']['labels']))
train_set_x = Transform(train_set, transforms)
train_batches = Batches(train_set_x, args.batch_size, shuffle=True, set_random_choices=True, num_workers=2)

test_set = list(zip(transpose(dataset['test']['data'] / 255.), dataset['test']['labels']))
test_batches = Batches(test_set, args.batch_size, shuffle=False, num_workers=2)

epsilon = (args.epsilon / 255.)
pgd_alpha = (args.pgd_alpha / 255.)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
# if args.model == 'PreActResNet18' or args.res18:
#     from learning.preactresnet import PreActResNet18SSL, PreActResNet18
#     model = PreActResNet18SSL() # TODO: make it single head?
#     c_head_model = Res18_out6_model()


# elif args.model == 'WideResNet':
#     # model = WideResNetMed_SSL(34, 10, widen_factor=args.width_factor, dropRate=0.0)
#     if args.carmon:
#         from learning.unlabel_WRN import WideResNet_2
#         model = WideResNet_2(depth=28, widen_factor=10)
#     elif args.TRADES or args.Bag:
#         from learning.unlabel_WRN import WideResNet_2
#         model = WideResNet_2(depth=34, widen_factor=10)

#     else:
#         model = WideResNetMed_SSL(34, 10, widen_factor=args.width_factor, dropRate=0.0)
#         # print(model)
#     c_head_model = WRN34_out_branch()

# else:
#     raise ValueError("Unknown model")

In [8]:
from Qmodels.utils import (upper_limit, lower_limit, std, mu, clamp, get_loaders,
    attack_pgd, evaluate_pgd, evaluate_standard)
from Qmodels.wide_resnet import WideResNet32
num_bits_list = [4,5,6,7,8,9,10,11,12,13,14,15,16]
_, _, dataset_normalization = get_loaders(args.data_dir, args.batch_size, dataset='cifar10', norm=False)
model = WideResNet32(num_bits_list, 10, normalize=dataset_normalization)
model = torch.nn.DataParallel(model)
pretrained_model = torch.load("Qmodels/model_wide_cifar10.pth")
partial = pretrained_model['state_dict']
model.load_state_dict(partial, strict=False)
c_head_model = WRN34_out_branch()
_ = model.cuda()

Files already downloaded and verified
Files already downloaded and verified


In [9]:
from torchsummary import summary
model.module.set_precision(num_bits=np.random.choice(num_bits_list), num_grad_bits=0)
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
NormalizeByChannelMeanStd-1            [-1, 3, 32, 32]               0
      QuantMeasure-2            [-1, 3, 32, 32]               0
           QConv2d-3           [-1, 16, 32, 32]             432
       BatchNorm2d-4           [-1, 16, 32, 32]              32
     USBatchNorm2d-5           [-1, 16, 32, 32]              32
              ReLU-6           [-1, 16, 32, 32]               0
      QuantMeasure-7           [-1, 16, 32, 32]               0
           QConv2d-8          [-1, 160, 32, 32]          23,040
       BatchNorm2d-9          [-1, 160, 32, 32]             320
    USBatchNorm2d-10          [-1, 160, 32, 32]             320
             ReLU-11          [-1, 160, 32, 32]               0
     QuantMeasure-12          [-1, 160, 32, 32]               0
          QConv2d-13          [-1, 160, 32, 32]         230,400
     QuantMeasure-14           [

In [10]:
# if not args.TRADES and not args.Bag:
    # model = nn.DataParallel(model).cuda()
c_head_model = nn.DataParallel(c_head_model).cuda()
c_head_model.train()


if args.l2:
    decay, no_decay = [], []
    for name, param in model.named_parameters():
        if 'bn' not in name and 'bias' not in name:
            decay.append(param)
        else:
            no_decay.append(param)
    params_bkbone = [{'params': decay, 'weight_decay': args.l2},
              {'params': no_decay, 'weight_decay': 0}]
else:
    # params_bkbone = model.parameters()
    decay, no_decay = [], []
    for name, param in c_head_model.named_parameters():
        if 'bn' not in name and 'bias' not in name:
            decay.append(param)
        else:
            no_decay.append(param)
    params = [{'params': decay, 'weight_decay': args.l2},
                     {'params': no_decay, 'weight_decay': 0}]

In [11]:
def lr_schedule(t):
    if t / args.epochs < 0.5:
        return args.lr_max
    elif t / args.epochs < 0.75:
        return args.lr_max / 10.
    else:
        return args.lr_max / 100.

In [12]:
learning_rate=1e-4
opt = torch.optim.Adam(params, lr=learning_rate)

s = 1
size = 32

In [13]:
from torchvision.transforms import transforms
# color_jitter = transforms.ColorJitter(0.8 * s, 0.8 * s, 0.8 * s, 0.2 * s)
transforms = torch.nn.Sequential(
    transforms.RandomResizedCrop(size=size),
    transforms.RandomHorizontalFlip(),
    # transforms.RandomApply([color_jitter], p=0.8),
    transforms.ColorJitter(0.8 * s, 0.8 * s, 0.8 * s, 0.2 * s),
    transforms.RandomGrayscale(p=0.2),
    # GaussianBlur(kernel_size=int(0.1 * size)),
)
scripted_transforms = torch.jit.script(transforms)
criterion = torch.nn.CrossEntropyLoss().cuda()

model.eval()
test_loss = 0
test_acc = 0
test_robust_loss = 0
test_robust_acc = 0
contrastive_attack_loss = 0
contrastive_clean_loss = 0

In [14]:
if args.res18:
    # import pdb; pdb.set_trace()
    if args.new:
        tmp = torch.load(args.ssl_model_path)['ssl_model']
    else:
        tmp = torch.load(args.ssl_model_path)
else:
    tmp = torch.load(args.ssl_model_path)['ssl_model']
c_head_model.load_state_dict(tmp)

c_head_model.eval()


for i, batch in enumerate(train_batches):
    model.module.set_precision(num_bits=np.random.choice(num_bits_list), num_grad_bits=0)
    X, y = batch['input'], batch['target']
    model.module.set_precision(num_bits=np.random.choice(num_bits_list), num_grad_bits=0)
    contrastive_Loss = \
        calculate_contrastive_Mhead_loss(X, scripted_transforms, model, criterion, c_head_model)
    break
    
# from autoattack import AutoAttack
from AAattack.autoattack import AutoAttack
adversary = AutoAttack(model, normalize, norm='Linf', eps=epsilon, log_path='./log/tmp.txt')
adversary.attacks_to_run = ['apgd-ce', 'apgd-t', 'fab-t', 'square']
adversary.apgd.n_restarts = 2
adversary.fab.n_restarts = 2

test_loss = 0
test_acc = 0
test_robust_loss = 0
test_robust_acc = 0
db_rob_acc_all=0
TestX = []
TestY = []
Testdelta = []
test_n = 0

db_test_acc_clean_all=0

print('epsilon', epsilon)



setting parameters for standard version
epsilon 0.03137254901960784


In [16]:
contrastive_attack_loss=0
contrastive_clean_loss=0
for i, batch in enumerate(test_batches):
    if i>9:#args.debug and i > 0:
        break
    
    X, y = batch['input'], batch['target']
    TestX.append(X)
    TestY.append(y)

    X = X.cuda()
    y = y.cuda()

    # Generate attack with AutoAttack
    adv_complete = adversary.run_standard_evaluation(X, y, bs=X.size(0))
    print('low', adv_complete.min(), adv_complete.max())
    delta = adv_complete - X
    print('delta', delta.min(), delta.max())

    with torch.no_grad():
        db_predict = model(normalize(adv_complete))[0]
        db_test_acc = (db_predict.max(1)[1] == y).sum().item()
        db_rob_acc_all += db_test_acc
        n = y.size(0)

        db_predict_clean = model(normalize(X))[0]
        db_test_acc_clean_all += (db_predict_clean.max(1)[1] == y).sum().item()
        print("db rob acc ", db_test_acc/n, '\n\n\n\n')

    # TODO: the saved adv is weaker than reported in AA paper, maybe of because rounding error?

    delta = delta.detach()

    Testdelta.append(delta)

    # import pdb; pdb.set_trace()

    with torch.no_grad():
        robust_output, _ = model(
            normalize(torch.clamp(X + delta[:X.size(0)], min=lower_limit, max=upper_limit)))
        output, _ = model(normalize(X))

        robust_loss = criterion(robust_output, y)
        loss = criterion(output, y)

        Adv_image = torch.clamp(X + delta[:X.size(0)], min=lower_limit, max=upper_limit)
        contrastive_attack = \
            calculate_contrastive_Mhead_loss(Adv_image, scripted_transforms, model, criterion, c_head_model, n_views=4)
        contrastive_clean = \
            calculate_contrastive_Mhead_loss(X, scripted_transforms, model, criterion, c_head_model, n_views=4)

    contrastive_attack_loss += contrastive_attack.item() * y.size(0)
    contrastive_clean_loss += contrastive_clean.item() * y.size(0)

    test_robust_loss += robust_loss.item() * y.size(0)
    test_robust_acc += (robust_output.max(1)[1] == y).sum().item()
    test_loss += loss.item() * y.size(0)
    test_acc += (output.max(1)[1] == y).sum().item()
    test_n += y.size(0)
    torch.cuda.empty_cache()
    print("test_robust_acc", test_robust_acc/test_n, db_rob_acc_all/test_n, 'clean', db_test_acc_clean_all/test_n)

print('clean contrastive=%.6f \t adv contrastive=%.6f' %
                    ((contrastive_clean_loss / test_n), (contrastive_attack_loss / test_n)))


using standard version including apgd-ce, apgd-t, fab-t, square
initial accuracy: 82.03%
apgd-ce - 1/1 - 38 out of 105 successfully perturbed
robust accuracy after APGD-CE: 52.34% (total time 170.0 s)
apgd-t - 1/1 - 6 out of 67 successfully perturbed
robust accuracy after APGD-T: 47.66% (total time 752.9 s)
fab-t - 1/1 - 0 out of 61 successfully perturbed
robust accuracy after FAB-T: 47.66% (total time 1485.5 s)
square - 1/1 - 0 out of 61 successfully perturbed
robust accuracy after SQUARE: 47.66% (total time 2341.0 s)
max Linf perturbation: 0.03137, nan in tensor: 0, max: 1.00000, min: 0.00000
robust accuracy: 47.66%
low tensor(0., device='cuda:0') tensor(1., device='cuda:0')
delta tensor(-0.0314, device='cuda:0') tensor(0.0314, device='cuda:0')
db rob acc  0.484375 




test_robust_acc 0.484375 0.484375 clean 0.8203125
using standard version including apgd-ce, apgd-t, fab-t, square
initial accuracy: 81.25%
apgd-ce - 1/1 - 41 out of 104 successfully perturbed
robust accuracy after APG

In [17]:
torch.save(TestX, "TestX.pt")
torch.save(TestY, "TestY.pt")
torch.save(Testdelta, "Testdelta.pt")
print(contrastive_attack_loss, contrastive_clean_loss)

7948.3343505859375 8128.197265625


In [14]:
TestX = torch.load("TestX.pt")
TestY = torch.load("TestY.pt")
Testdelta = torch.load( "Testdelta.pt")
# TestX = TestX[11:]
# TestY = TestY[11:]
# Testdelta = Testdelta[11:]
contrastive_attack_loss = 7948.3343505859375
contrastive_clean_loss = 8128.197265625

In [17]:
(test_robust_ada_acc, test_clean_ada_acc,test_n)

(783, 1024, 1280)

In [18]:
((test_loss), (test_acc), (test_clean_ada_acc / test_n * 100))

(0, 0, 80.0)

In [19]:
torch.save(test_robust_ada_acc, "test_robust_ada_acc.pt")

In [15]:
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = False

In [16]:
TestX = torch.cat(TestX, dim=0)
TestY = torch.cat(TestY, dim=0)
Testdelta = torch.cat(Testdelta, dim=0)
print(TestX.size(), TestY.size(), Testdelta.size())

total_len = TestX.shape[0]
ind = [i for i in range(total_len)]
from random import shuffle
shuffle(ind)
# TestX = TestX[ind]
# TestY = TestY[ind]
# Testdelta = Testdelta[ind]

bs = 64 #args.contrastive_bs

num_bs = TestX.size(0) // bs
if num_bs * bs < TestX.size(0):
    num_bs += 1
count_test = 0
print(num_bs)
Base_atack_steps = [20]
for base_attack_step in Base_atack_steps:
    for adda_times in [2]:
        test_robust_ada_acc = 0
        test_clean_ada_acc = 0
        test_robust_ada_loss = 0
        test_clean_ada_loss = 0
        # test_clean_loss = 0
        # test_clean_acc = 0

        test_n = 0
        adaadv_contrastive_loss=0

        print('contrastive bs', bs)

        for bs_ind in range(num_bs):
            print(bs_ind)
            if args.debug and bs_ind > 0:
                break
            model.module.set_precision(num_bits=np.random.choice(num_bits_list), num_grad_bits=0)
            X = TestX[bs_ind * bs:(bs_ind + 1) * bs]
            y = TestY[bs_ind * bs:(bs_ind + 1) * bs]
            delta = Testdelta[bs_ind * bs:(bs_ind + 1) * bs]

            X = X.cuda()
            y = y.cuda()
            delta = delta.cuda()

            # Random initialization
            if args.random_noise:
                delta2 = torch.zeros_like(X)
                delta2.uniform_(-epsilon* adda_times, epsilon* adda_times)
            else:
                delta2 = attack_constrastive_Mhead(model, c_head_model, scripted_transforms, criterion,
                                                   torch.clamp(X + delta[:X.size(0)], min=lower_limit, max=upper_limit),
                                                   torch.zeros_like(y), epsilon * adda_times, pgd_alpha,  # 1, 0.2,
                                                   int(base_attack_step * adda_times) if not args.rand else 0,
                                                   args.restarts, args.norm, n_views=args.n_views
                                                   )
                delta2 = delta2.detach()
            model.module.set_precision(num_bits=np.random.choice(num_bits_list), num_grad_bits=0)
            robust_output_ada, hidden = model(
                normalize(torch.clamp(X + delta[:X.size(0)], min=lower_limit, max=upper_limit) + delta2))
            test_robust_ada_acc += (robust_output_ada.max(1)[1] == y).sum().item()
            robust_ada_loss = criterion(robust_output_ada, y)
            test_robust_ada_loss += robust_ada_loss.item() * y.size(0)

            torch.cuda.empty_cache()

            contrastive_ada_attack = \
                calculate_contrastive_Mhead_loss(
                    torch.clamp(
                        torch.clamp(X + delta[:X.size(0)], min=lower_limit, max=upper_limit) + delta2,
                        min=lower_limit, max=upper_limit),
                    scripted_transforms, model, criterion, c_head_model)
            adaadv_contrastive_loss += contrastive_ada_attack.item() * y.size(0)

            torch.cuda.empty_cache()

            if args.random_noise:
                delta3 = delta2
            else:
                delta3 = attack_constrastive_Mhead(model, c_head_model, scripted_transforms,
                                                   criterion,
                                                   X,
                                                   torch.zeros_like(y), epsilon * adda_times, pgd_alpha,  # 1, 0.2,
                                                   int(base_attack_step * adda_times) if not args.rand else 0,
                                                   args.restarts, args.norm,
                                                   early_stop=args.eval, n_views=args.n_views)
                #     #epsilon * args.adda_times, pgd_alpha
                delta3 = delta3.detach()

            clean_output_ada, hidden = model(
                normalize(X + delta3))
            test_clean_ada_acc += (clean_output_ada.max(1)[1] == y).sum().item()

            clean_ada_loss = criterion(clean_output_ada, y)
            test_clean_ada_loss += clean_ada_loss.item() * y.size(0)
            test_n += y.size(0)
            # print(test_robust_ada_acc, test_robust_ada_loss)
            print(
            'e=%d  scale=%.4f step=%d epsilon=%d \t TestLoss=%.4f TestAcc=%.4f TestCleanAdaAcc=%.4f \t TestRobLoss=%.4f TestRobAcc %.4f \t AdaTestLoss=%.4f AdaTestAcc %.4f' %
            (0, adda_times, (int(base_attack_step * adda_times)), (epsilon*255),
             (test_loss / test_n), (test_acc / test_n * 100), (test_clean_ada_acc / test_n * 100),
             (test_robust_loss / test_n), (test_robust_acc / test_n * 100),
             (test_robust_loss / test_n), (test_robust_ada_acc / test_n * 100)))
            torch.cuda.empty_cache()
            # print(bs_ind)

        print("Final: " +  
            'e=%d  scale=%.4f step=%d epsilon=%d \t TestLoss=%.4f TestAcc=%.4f TestCleanAdaAcc=%.4f \t TestRobLoss=%.4f TestRobAcc %.4f \t AdaTestLoss=%.4f AdaTestAcc %.4f' %
            (0, adda_times, (int(base_attack_step * adda_times)), (epsilon*255),
             (test_loss / test_n), (test_acc / test_n * 100), (test_clean_ada_acc / test_n * 100),
             (test_robust_loss / test_n), (test_robust_acc / test_n * 100),
             (test_robust_loss / test_n), (test_robust_ada_acc / test_n * 100)))
        print('clean contrastive=%.6f \t adv contrastive=%.6f \t adaadv contrastive=%.6f' %
              ((contrastive_clean_loss / test_n), (contrastive_attack_loss / test_n),
               (adaadv_contrastive_loss / test_n)))
print("\n\n\nmogu")

torch.Size([1280, 3, 32, 32]) torch.Size([1280]) torch.Size([1280, 3, 32, 32])
20
contrastive bs 64
0
e=0  scale=2.0000 step=40 epsilon=8 	 TestLoss=0.0000 TestAcc=0.0000 TestCleanAdaAcc=73.4375 	 TestRobLoss=0.0000 TestRobAcc 0.0000 	 AdaTestLoss=0.0000 AdaTestAcc 53.1250
1
e=0  scale=2.0000 step=40 epsilon=8 	 TestLoss=0.0000 TestAcc=0.0000 TestCleanAdaAcc=79.6875 	 TestRobLoss=0.0000 TestRobAcc 0.0000 	 AdaTestLoss=0.0000 AdaTestAcc 58.5938
2
e=0  scale=2.0000 step=40 epsilon=8 	 TestLoss=0.0000 TestAcc=0.0000 TestCleanAdaAcc=81.2500 	 TestRobLoss=0.0000 TestRobAcc 0.0000 	 AdaTestLoss=0.0000 AdaTestAcc 60.9375
3
e=0  scale=2.0000 step=40 epsilon=8 	 TestLoss=0.0000 TestAcc=0.0000 TestCleanAdaAcc=82.0312 	 TestRobLoss=0.0000 TestRobAcc 0.0000 	 AdaTestLoss=0.0000 AdaTestAcc 60.9375
4
e=0  scale=2.0000 step=40 epsilon=8 	 TestLoss=0.0000 TestAcc=0.0000 TestCleanAdaAcc=80.3125 	 TestRobLoss=0.0000 TestRobAcc 0.0000 	 AdaTestLoss=0.0000 AdaTestAcc 59.3750
5
e=0  scale=2.0000 step=40 ep

In [1]:
import torch
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(t,r,a,f)

17071734784 0 0 0
